#### This script works on transform the seq2seq model to a graphical model using antra

In [1]:
import os
import sys
import json
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

from model import *
from ReaSCAN_dataset import *

In [2]:
data_directory = "../../../data-files/ReaSCAN-Simple/"
data_file = "data-compositional-splits.txt"
input_vocab_file = "input_vocabulary.txt"
target_vocab_file = "target_vocabulary.txt"

In [3]:
dataset = ReaSCANDataset(
    json.load(open(os.path.join(data_directory, data_file), "r")), 
    data_directory, split="train",
    input_vocabulary_file=input_vocab_file,
    target_vocabulary_file=target_vocab_file,
    generate_vocabulary=False,
    k=0,
)

2021-07-08 20:57 Formulating the dataset from the passed in json file...
2021-07-08 20:57 Loading vocabularies...
2021-07-08 20:57 Done loading vocabularies.


In [4]:
model = Model(
    input_vocabulary_size=dataset.input_vocabulary_size,
    target_vocabulary_size=dataset.target_vocabulary_size,
    num_cnn_channels=dataset.image_channels,
    input_padding_idx=dataset.input_vocabulary.pad_idx,
    target_pad_idx=dataset.target_vocabulary.pad_idx,
    target_eos_idx=dataset.target_vocabulary.eos_idx,
    # language encoder config
    embedding_dimension=25,
    encoder_hidden_size=100,
    num_encoder_layers=1,
    encoder_dropout_p=0.3,
    encoder_bidirectional=True,
    # world encoder config
    simple_situation_representation=True,
    cnn_hidden_num_channels=50,
    cnn_kernel_size=7,
    cnn_dropout_p=0.1,
    auxiliary_task=False,
    # decoder config
    num_decoder_layers=1,
    attention_type="bahdanau",
    decoder_dropout_p=0.3,
    decoder_hidden_size=100,
    conditional_attention=True,
    output_directory="../../../saved_models/ReaSCAN-Simple/"
)

/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [5]:
model

Model(
  (situation_encoder): ConvolutionalNet(
    (conv_1): Conv2d(3, 50, kernel_size=(1, 1), stride=(1, 1))
    (conv_2): Conv2d(3, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv_3): Conv2d(3, 50, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (dropout): Dropout(p=0.1, inplace=False)
    (relu): ReLU()
    (layers): Sequential(
      (0): ReLU()
      (1): Dropout(p=0.1, inplace=False)
    )
  )
  (visual_attention): Attention(
    (key_layer): Linear(in_features=150, out_features=100, bias=False)
    (query_layer): Linear(in_features=100, out_features=100, bias=False)
    (energy_layer): Linear(in_features=100, out_features=1, bias=False)
  )
  (encoder): EncoderRNN(
    EncoderRNN
     bidirectional=True 
     num_layers=1
     hidden_size=100
     dropout=0.3
     n_input_symbols=20
    
    (embedding): Embedding(20, 25, padding_idx=0)
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm): LSTM(25, 100, dropout=0.3, bidirectional=True)
  )
  (enc_hidd